In [3]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [4]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [5]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [6]:
def Act(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Act', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [7]:
def Env(states, actions, state_size, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Env', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        states_logits = tf.layers.dense(inputs=nl2, units=state_size, trainable=False)
        Qlogits = tf.layers.dense(inputs=nl2, units=1, trainable=False)
        return states_logits, Qlogits

In [8]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, rewards, dones, rates):
    ################################################ a = act(s)
    actions_logits = Act(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ################################################ s', r = env(s, a)
    ################################################ s', Q = env(s, a)
    ################################################ ~s', ~Q = env(s, ~a)
    e_next_states_logits, eQs = Env(actions=actions_labels, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size)
    a_next_states_logits, aQs = Env(actions=actions_logits, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size, reuse=True)
    next_states_labels = tf.nn.sigmoid(next_states)
    eloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=e_next_states_logits, 
                                                                   labels=next_states_labels))
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                    labels=next_states_labels))
    eQs_logits = tf.reshape(eQs, shape=[-1])
    aQs_logits = tf.reshape(aQs, shape=[-1])
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=eQs_logits, # GAN
                                                                    labels=rates)) # 0-1
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs_logits, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs_logits, # GAN
                                                                     labels=tf.ones_like(rates))) # max
    #################################################### s'', Q' = ~env(s', ~a')
    next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    _, aQs2 = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                  action_size=action_size, state_size=state_size, reuse=True)
    aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
                                                                     labels=tf.ones_like(rates))) # max
    ###################################################### Q(s,a)= r + Q'(s',a') # max
    ###################################################### ~Q(s,~a)= r # min
    ###################################################### ~Q(s,~a)= r + Q'(s',a') # max
    targetQs = rewards + (gamma * aQs2_logits)
    eloss += tf.reduce_mean(tf.square(eQs_logits - targetQs)) # max
    eloss += tf.reduce_mean(tf.square(aQs_logits - rewards)) # min
    aloss2 += tf.reduce_mean(tf.square(aQs_logits - targetQs)) # max
    return actions_logits, aloss, eloss, aloss2

In [9]:
def model_opt(a_loss, e_loss, a_loss2, a_learning_rate, e_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('Act')]
    e_vars = [var for var in t_vars if var.name.startswith('Env')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        e_opt = tf.train.AdamOptimizer(e_learning_rate).minimize(e_loss, var_list=e_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, e_opt, a_opt2

In [10]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, e_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.e_loss, self.a_loss2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, 
            rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.e_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        e_loss=self.e_loss,
                                                        a_loss2=self.a_loss2, 
                                                        a_learning_rate=a_learning_rate,
                                                        e_learning_rate=e_learning_rate)

In [12]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch
#     def sample(self, batch_size):
#         idx = np.random.choice(np.arange(len(self.buffer)), size=batch_size, replace=False)
#         return [self.buffer[ii] for ii in idx]

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [13]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [15]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
e_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size
gamma=0.99

In [16]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size, gamma=gamma,
              a_learning_rate=a_learning_rate, e_learning_rate=e_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [17]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        state = env.reset()
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
aloss_list, eloss_list, aloss2_list = [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes for running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        aloss_batch, eloss_batch, aloss2_batch = [], [], []
        total_reward = 0
        state = env.reset()
        num_step = 0
        rate = -1

        # Training steps/batches
        while True:
            # Explore (env) or Exploit (model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            while True:
                idx = np.random.choice(np.arange(memory_size// batch_size))
                states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                states = states[rates >= np.max(rates)]
                actions = actions[rates >= np.max(rates)]
                next_states = next_states[rates >= np.max(rates)]
                rewards = rewards[rates >= np.max(rates)]
                dones = dones[rates >= np.max(rates)]
                rates = rates[rates >= np.max(rates)]
                if np.count_nonzero(dones) > 0 and len(dones) > 1 and np.max(rates) > 0:
                    break
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            eloss, _ = sess.run([model.e_loss, model.e_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss2, _= sess.run([model.a_loss2, model.a_opt2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            # print(len(dones), np.count_nonzero(dones), np.max(rates))
            aloss_batch.append(aloss)
            eloss_batch.append(eloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        # Rating the latest played episode
        rate = total_reward/500 # update rate at the end/ when episode is done
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                memory.buffer[-1-idx][-1] = rate # rate the trajectory/data

        # Print out
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'eloss:{:.4f}'.format(np.mean(eloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        aloss_list.append([ep, np.mean(aloss_batch)])
        eloss_list.append([ep, np.mean(eloss_batch)])
        aloss2_list.append([ep, np.mean(aloss2_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:16.0000 R:16.0000 rate:0.0320 aloss:0.6940 eloss:4.3616 aloss2:3.0810 exploreP:0.9984
Episode:1 meanR:19.0000 R:22.0000 rate:0.0440 aloss:0.6965 eloss:4.3686 aloss2:3.0711 exploreP:0.9962
Episode:2 meanR:21.3333 R:26.0000 rate:0.0520 aloss:0.6949 eloss:4.3494 aloss2:3.0587 exploreP:0.9937
Episode:3 meanR:20.0000 R:16.0000 rate:0.0320 aloss:0.6961 eloss:4.3551 aloss2:3.0367 exploreP:0.9921
Episode:4 meanR:19.8000 R:19.0000 rate:0.0380 aloss:0.6933 eloss:4.3458 aloss2:3.0398 exploreP:0.9902
Episode:5 meanR:20.3333 R:23.0000 rate:0.0460 aloss:0.6922 eloss:4.3354 aloss2:3.0202 exploreP:0.9880
Episode:6 meanR:21.1429 R:26.0000 rate:0.0520 aloss:0.6920 eloss:4.3403 aloss2:3.0161 exploreP:0.9855
Episode:7 meanR:20.6250 R:17.0000 rate:0.0340 aloss:0.6902 eloss:4.3457 aloss2:3.0058 exploreP:0.9838
Episode:8 meanR:22.1111 R:34.0000 rate:0.0680 aloss:0.6903 eloss:4.3325 aloss2:2.9986 exploreP:0.9805
Episode:9 meanR:21.2000 R:13.0000 rate:0.0260 aloss:0.6925 eloss:4.3271 aloss2:2.9

Episode:80 meanR:27.3457 R:24.0000 rate:0.0480 aloss:0.6740 eloss:5.4519 aloss2:2.6319 exploreP:0.8033
Episode:81 meanR:27.5488 R:44.0000 rate:0.0880 aloss:0.6851 eloss:5.4343 aloss2:2.6441 exploreP:0.7998
Episode:82 meanR:27.6145 R:33.0000 rate:0.0660 aloss:0.6720 eloss:5.4099 aloss2:2.6462 exploreP:0.7972
Episode:83 meanR:27.6429 R:30.0000 rate:0.0600 aloss:0.6918 eloss:5.4059 aloss2:2.6406 exploreP:0.7949
Episode:84 meanR:27.4824 R:14.0000 rate:0.0280 aloss:0.6729 eloss:5.3549 aloss2:2.6634 exploreP:0.7938
Episode:85 meanR:27.3953 R:20.0000 rate:0.0400 aloss:0.6838 eloss:5.3557 aloss2:2.6634 exploreP:0.7922
Episode:86 meanR:27.9195 R:73.0000 rate:0.1460 aloss:0.6864 eloss:5.3323 aloss2:2.6724 exploreP:0.7865
Episode:87 meanR:27.7841 R:16.0000 rate:0.0320 aloss:0.6961 eloss:5.2573 aloss2:2.6752 exploreP:0.7853
Episode:88 meanR:27.9551 R:43.0000 rate:0.0860 aloss:0.6793 eloss:5.2828 aloss2:2.6910 exploreP:0.7819
Episode:89 meanR:27.7889 R:13.0000 rate:0.0260 aloss:0.6739 eloss:5.3037 

Episode:159 meanR:30.7500 R:39.0000 rate:0.0780 aloss:0.6909 eloss:4.4984 aloss2:3.1939 exploreP:0.6362
Episode:160 meanR:30.5400 R:9.0000 rate:0.0180 aloss:0.7006 eloss:4.5086 aloss2:3.1828 exploreP:0.6356
Episode:161 meanR:30.6900 R:27.0000 rate:0.0540 aloss:0.7084 eloss:4.5009 aloss2:3.1906 exploreP:0.6339
Episode:162 meanR:30.4400 R:11.0000 rate:0.0220 aloss:0.8404 eloss:4.4820 aloss2:3.1586 exploreP:0.6332
Episode:163 meanR:30.2600 R:12.0000 rate:0.0240 aloss:0.7293 eloss:4.4915 aloss2:3.1904 exploreP:0.6325
Episode:164 meanR:30.1300 R:17.0000 rate:0.0340 aloss:0.7308 eloss:4.4978 aloss2:3.1833 exploreP:0.6314
Episode:165 meanR:30.2600 R:29.0000 rate:0.0580 aloss:0.7442 eloss:4.4887 aloss2:3.1798 exploreP:0.6296
Episode:166 meanR:30.3000 R:44.0000 rate:0.0880 aloss:0.7359 eloss:4.4908 aloss2:3.1832 exploreP:0.6269
Episode:167 meanR:30.7300 R:60.0000 rate:0.1200 aloss:0.7358 eloss:4.4937 aloss2:3.1798 exploreP:0.6232
Episode:168 meanR:30.8500 R:39.0000 rate:0.0780 aloss:0.7024 elos

Episode:238 meanR:27.9000 R:55.0000 rate:0.1100 aloss:0.6918 eloss:4.5935 aloss2:3.0176 exploreP:0.5114
Episode:239 meanR:27.9700 R:31.0000 rate:0.0620 aloss:0.7059 eloss:4.6058 aloss2:3.0057 exploreP:0.5098
Episode:240 meanR:28.0200 R:27.0000 rate:0.0540 aloss:0.6958 eloss:4.6091 aloss2:3.0047 exploreP:0.5085
Episode:241 meanR:27.9200 R:15.0000 rate:0.0300 aloss:0.6706 eloss:4.6083 aloss2:3.0064 exploreP:0.5077
Episode:242 meanR:28.1100 R:47.0000 rate:0.0940 aloss:0.6913 eloss:4.5994 aloss2:2.9929 exploreP:0.5054
Episode:243 meanR:28.4000 R:38.0000 rate:0.0760 aloss:0.6880 eloss:4.6021 aloss2:2.9960 exploreP:0.5035
Episode:244 meanR:28.5200 R:47.0000 rate:0.0940 aloss:0.6770 eloss:4.5979 aloss2:2.9965 exploreP:0.5012
Episode:245 meanR:28.7800 R:59.0000 rate:0.1180 aloss:0.6818 eloss:4.5999 aloss2:2.9890 exploreP:0.4983
Episode:246 meanR:28.7400 R:17.0000 rate:0.0340 aloss:0.6898 eloss:4.6007 aloss2:2.9866 exploreP:0.4975
Episode:247 meanR:28.7900 R:27.0000 rate:0.0540 aloss:0.6790 elo

Episode:317 meanR:44.0900 R:58.0000 rate:0.1160 aloss:0.7616 eloss:4.7079 aloss2:2.8924 exploreP:0.3479
Episode:318 meanR:44.4100 R:48.0000 rate:0.0960 aloss:0.7274 eloss:4.6945 aloss2:2.8969 exploreP:0.3463
Episode:319 meanR:44.9800 R:78.0000 rate:0.1560 aloss:0.7576 eloss:4.7081 aloss2:2.8899 exploreP:0.3437
Episode:320 meanR:45.1700 R:33.0000 rate:0.0660 aloss:0.8214 eloss:4.7338 aloss2:2.8809 exploreP:0.3426
Episode:321 meanR:45.5100 R:43.0000 rate:0.0860 aloss:0.7427 eloss:4.7045 aloss2:2.8888 exploreP:0.3412
Episode:322 meanR:46.1400 R:78.0000 rate:0.1560 aloss:0.7769 eloss:4.7163 aloss2:2.8885 exploreP:0.3386
Episode:323 meanR:46.9100 R:98.0000 rate:0.1960 aloss:0.7706 eloss:4.7149 aloss2:2.8905 exploreP:0.3354
Episode:324 meanR:47.0800 R:35.0000 rate:0.0700 aloss:0.7766 eloss:4.7025 aloss2:2.8920 exploreP:0.3342
Episode:325 meanR:47.2900 R:41.0000 rate:0.0820 aloss:0.7860 eloss:4.7125 aloss2:2.8931 exploreP:0.3329
Episode:326 meanR:47.6400 R:55.0000 rate:0.1100 aloss:0.7508 elo

...

Episode:873 meanR:109.8200 R:132.0000 rate:0.2640 aloss:0.7416 eloss:4.5568 aloss2:2.8664 exploreP:0.0634
Episode:874 meanR:111.3900 R:222.0000 rate:0.4440 aloss:0.7081 eloss:4.5425 aloss2:2.8717 exploreP:0.0622
Episode:875 meanR:111.8600 R:165.0000 rate:0.3300 aloss:0.7094 eloss:4.5334 aloss2:2.8711 exploreP:0.0614
Episode:876 meanR:112.3700 R:192.0000 rate:0.3840 aloss:0.7023 eloss:4.5333 aloss2:2.8730 exploreP:0.0604
Episode:877 meanR:112.5100 R:142.0000 rate:0.2840 aloss:0.7043 eloss:4.5325 aloss2:2.8757 exploreP:0.0597
Episode:878 meanR:113.4600 R:162.0000 rate:0.3240 aloss:0.7261 eloss:4.5309 aloss2:2.8687 exploreP:0.0589
Episode:879 meanR:113.8700 R:136.0000 rate:0.2720 aloss:0.7584 eloss:4.5458 aloss2:2.8646 exploreP:0.0582
Episode:880 meanR:114.2500 R:109.0000 rate:0.2180 aloss:0.7227 eloss:4.5422 aloss2:2.8705 exploreP:0.0577
Episode:881 meanR:114.7600 R:126.0000 rate:0.2520 aloss:0.7205 eloss:4.5483 aloss2:2.8685 exploreP:0.0571
Episode:882 meanR:115.0200 R:94.0000 rate:0.18

Episode:951 meanR:109.4000 R:10.0000 rate:0.0200 aloss:0.7197 eloss:4.4460 aloss2:2.8938 exploreP:0.0357
Episode:952 meanR:107.6500 R:55.0000 rate:0.1100 aloss:0.7372 eloss:4.4691 aloss2:2.8844 exploreP:0.0356
Episode:953 meanR:104.4300 R:10.0000 rate:0.0200 aloss:0.7070 eloss:4.4431 aloss2:2.8792 exploreP:0.0356
Episode:954 meanR:103.2600 R:10.0000 rate:0.0200 aloss:0.6923 eloss:4.4846 aloss2:2.8920 exploreP:0.0356
Episode:955 meanR:102.0100 R:55.0000 rate:0.1100 aloss:0.7019 eloss:4.4750 aloss2:2.8886 exploreP:0.0354
Episode:956 meanR:99.9100 R:10.0000 rate:0.0200 aloss:0.7588 eloss:4.4607 aloss2:2.8722 exploreP:0.0354
Episode:957 meanR:98.7400 R:79.0000 rate:0.1580 aloss:0.6953 eloss:4.4673 aloss2:2.8770 exploreP:0.0352
Episode:958 meanR:97.9400 R:65.0000 rate:0.1300 aloss:0.6959 eloss:4.4755 aloss2:2.8662 exploreP:0.0350
Episode:959 meanR:97.0500 R:82.0000 rate:0.1640 aloss:0.7011 eloss:4.5303 aloss2:2.8355 exploreP:0.0348
Episode:960 meanR:96.2200 R:73.0000 rate:0.1460 aloss:0.719

Episode:1030 meanR:78.5200 R:321.0000 rate:0.6420 aloss:0.6690 eloss:4.5020 aloss2:2.8589 exploreP:0.0227
Episode:1031 meanR:79.0800 R:125.0000 rate:0.2500 aloss:0.6615 eloss:4.5041 aloss2:2.8635 exploreP:0.0226
Episode:1032 meanR:81.3000 R:283.0000 rate:0.5660 aloss:0.6887 eloss:4.5099 aloss2:2.8596 exploreP:0.0222
Episode:1033 meanR:81.6400 R:94.0000 rate:0.1880 aloss:0.6616 eloss:4.4923 aloss2:2.8661 exploreP:0.0221
Episode:1034 meanR:81.5000 R:62.0000 rate:0.1240 aloss:0.7274 eloss:4.5100 aloss2:2.8658 exploreP:0.0220
Episode:1035 meanR:81.8000 R:92.0000 rate:0.1840 aloss:0.6968 eloss:4.5015 aloss2:2.8692 exploreP:0.0219
Episode:1036 meanR:81.6900 R:72.0000 rate:0.1440 aloss:0.6858 eloss:4.4957 aloss2:2.8712 exploreP:0.0218
Episode:1037 meanR:81.6900 R:59.0000 rate:0.1180 aloss:0.6773 eloss:4.5123 aloss2:2.8671 exploreP:0.0218
Episode:1038 meanR:82.3400 R:118.0000 rate:0.2360 aloss:0.6784 eloss:4.5121 aloss2:2.8671 exploreP:0.0216
Episode:1039 meanR:82.9100 R:108.0000 rate:0.2160 a

Episode:1108 meanR:93.1600 R:55.0000 rate:0.1100 aloss:0.6452 eloss:4.5235 aloss2:2.8497 exploreP:0.0167
Episode:1109 meanR:93.2400 R:66.0000 rate:0.1320 aloss:0.6479 eloss:4.5212 aloss2:2.8534 exploreP:0.0166
Episode:1110 meanR:93.3700 R:84.0000 rate:0.1680 aloss:0.6310 eloss:4.5292 aloss2:2.8560 exploreP:0.0166
Episode:1111 meanR:92.9300 R:45.0000 rate:0.0900 aloss:0.6856 eloss:4.5297 aloss2:2.8625 exploreP:0.0166
Episode:1112 meanR:92.4300 R:47.0000 rate:0.0940 aloss:0.6812 eloss:4.5146 aloss2:2.8593 exploreP:0.0165
Episode:1113 meanR:91.6400 R:44.0000 rate:0.0880 aloss:0.6344 eloss:4.5164 aloss2:2.8640 exploreP:0.0165
Episode:1114 meanR:91.2200 R:76.0000 rate:0.1520 aloss:0.6655 eloss:4.5073 aloss2:2.8548 exploreP:0.0164
Episode:1115 meanR:89.4200 R:44.0000 rate:0.0880 aloss:0.6401 eloss:4.4955 aloss2:2.8606 exploreP:0.0164
Episode:1116 meanR:88.8800 R:80.0000 rate:0.1600 aloss:0.6458 eloss:4.5130 aloss2:2.8532 exploreP:0.0164
Episode:1117 meanR:88.3200 R:68.0000 rate:0.1360 aloss:

Episode:1186 meanR:64.2700 R:86.0000 rate:0.1720 aloss:0.6350 eloss:4.5304 aloss2:2.8516 exploreP:0.0141
Episode:1187 meanR:63.8100 R:44.0000 rate:0.0880 aloss:0.6352 eloss:4.5067 aloss2:2.8599 exploreP:0.0141
Episode:1188 meanR:63.5800 R:62.0000 rate:0.1240 aloss:0.6360 eloss:4.5251 aloss2:2.8440 exploreP:0.0140
Episode:1189 meanR:63.3600 R:41.0000 rate:0.0820 aloss:0.6158 eloss:4.5101 aloss2:2.8566 exploreP:0.0140
Episode:1190 meanR:64.1700 R:121.0000 rate:0.2420 aloss:0.6382 eloss:4.5478 aloss2:2.8511 exploreP:0.0140
Episode:1191 meanR:64.1400 R:60.0000 rate:0.1200 aloss:0.6230 eloss:4.5232 aloss2:2.8507 exploreP:0.0140
Episode:1192 meanR:64.1800 R:51.0000 rate:0.1020 aloss:0.6400 eloss:4.5124 aloss2:2.8547 exploreP:0.0139
Episode:1193 meanR:63.3700 R:51.0000 rate:0.1020 aloss:0.6474 eloss:4.5285 aloss2:2.8489 exploreP:0.0139
Episode:1194 meanR:63.6900 R:84.0000 rate:0.1680 aloss:0.6053 eloss:4.5254 aloss2:2.8577 exploreP:0.0139
Episode:1195 meanR:63.5700 R:60.0000 rate:0.1200 aloss

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(aloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses')

In [ ]:
eps, arr = np.array(eloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Env losses')

In [ ]:
eps, arr = np.array(aloss2_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses 2')

## Testing

Let's checkout how our trained agent plays the game.

In [24]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
